In [18]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [19]:
df1 = pd.read_csv("Data/PreProcessedData.csv")
df_no_plot = df1.drop('Plot', axis = 1)
df2 = pd.read_csv("Data/LDA_topics.csv") #UPDATE THIS TO BE WHATEVER DATA WE GIVE IT
df_LDATOPICS = pd.concat([df_no_plot, df2], axis=1)
df_LDATOPICS = df_LDATOPICS.drop('Title', axis = 1)
print(df_LDATOPICS)

      IMDbRating  Year  Action  Adult  Adventure  Animation  Biography  \
0            7.1  2000       0      0          0          0          0   
1            4.1  2000       0      0          0          0          0   
2            6.6  2000       0      0          1          0          0   
3            5.6  2000       0      0          0          0          0   
4            7.7  2000       0      0          0          0          0   
...          ...   ...     ...    ...        ...        ...        ...   
6509         4.0  2022       0      0          0          0          0   
6510         7.9  2022       0      0          1          1          0   
6511         7.6  2022       0      0          0          0          0   
6512         6.9  2022       0      0          0          0          1   
6513         6.7  2022       0      0          0          0          0   

      Comedy  Crime  Documentary  ...  Unnamed: 0        t1        t2  \
0          0      0            0  ... 

In [24]:
df2 = pd.read_csv("Data/LDA_topics_synonym.csv")
df_LDATOPICS_synonym = pd.concat([df_no_plot, df2], axis=1)
df_LDATOPICS_synonym = df_LDATOPICS_synonym.drop('Title', axis = 1)

In [25]:
def test_trainsets(df):
    rating = df['IMDbRating']
    xdf = df.drop('IMDbRating', axis=1, inplace=False)
    
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(xdf, rating, test_size=0.2, random_state=42)
    
    X_train_df = pd.DataFrame(X_train)
    X_test_df = pd.DataFrame(X_test)
    y_train_df = pd.DataFrame(y_train)
    y_test_df = pd.DataFrame(y_test)

    return X_train_df, y_train_df, X_test_df, y_test_df

In [ ]:
#ROOM FOR PARAMETER TESTING

In [27]:
X_train, y_train, X_test, y_test = test_trainsets(df_LDATOPICS)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Kfold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# store the score for each fold
scores = []

mae_scores = []
baseline_scores = []
# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training data
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation data
    y_pred_fold = xgb_model.predict(X_val_fold)
    # Compute the MAE score
    baseline = mean_absolute_error(y_val_fold, np.array([np.mean(y_train['IMDbRating'])] * len(y_pred_fold)))
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    print(mae)
    print(baseline)
    mae_scores.append(mae)
    baseline_scores.append(baseline)

# Compute the average MAE score
average_mae = sum(mae_scores) / len(mae_scores)
average_baseline = sum(baseline_scores) / len(baseline_scores)

y_pred = xgb_model.predict(X_test)
print(y_pred)
print(y_test)
results = pd.DataFrame({'Actual': y_test['IMDbRating'], 'Prediction': y_pred})
results.to_csv('LDA_Topics_results.csv', index=False)

0.678209176159545
0.776840421462601
0.732682787022069
0.8239868527045442
0.7345483812855668
0.8302755760643642
0.6901474795277426
0.7796659473113684
0.7152102596700306
0.8062892206100271
[6.53644   6.2246327 6.232986  ... 5.810945  5.52865   5.972417 ]
      IMDbRating
5932         5.7
1872         6.3
1002         6.8
6036         3.1
5421         7.0
...          ...
6367         6.2
4560         6.5
4152         4.6
3964         6.2
1844         7.4

[1303 rows x 1 columns]


In [26]:
X_train, y_train, X_test, y_test = test_trainsets(df_LDATOPICS_synonym)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Kfold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# store the score for each fold
scores = []

mae_scores = []
baseline_scores = []
# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training data
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation data
    y_pred_fold = xgb_model.predict(X_val_fold)
    # Compute the MAE score
    baseline = mean_absolute_error(y_val_fold, np.array([np.mean(y_train['IMDbRating'])] * len(y_pred_fold)))
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    print(mae)
    print(baseline)
    mae_scores.append(mae)
    baseline_scores.append(baseline)

# Compute the average MAE score
average_mae = sum(mae_scores) / len(mae_scores)
average_baseline = sum(baseline_scores) / len(baseline_scores)

y_pred = xgb_model.predict(X_test)
print(y_pred)
print(y_test)
results = pd.DataFrame({'Actual': y_test['IMDbRating'], 'Prediction': y_pred})
results.to_csv('LDA_Topics_synonym_results.csv', index=False)

0.6905952620026249
0.776840421462601
0.7194052845868862
0.8239868527045442
0.7220759663609305
0.8302755760643642
0.6922737716484436
0.7796659473113684
0.7190202249446437
0.8062892206100271
[6.6005907 6.641737  6.3465843 ... 6.058514  5.7030725 6.295929 ]
      IMDbRating
5932         5.7
1872         6.3
1002         6.8
6036         3.1
5421         7.0
...          ...
6367         6.2
4560         6.5
4152         4.6
3964         6.2
1844         7.4

[1303 rows x 1 columns]
